# Alpha de Jensen en Python

- Partiendo de la base del ejercicio en R "Ejercicio 2-6 Vectoriza todo el programa", se realiza su migracion a Python,
- Se intenta mejorar al máximo el rendimiento

In [1]:
import pandas as pd
import numpy as np
from time import time
from timeit import timeit

In [2]:
tiempo_inicial = time()

window_width=30

# Cargamos los datos y extraemos por un lado los activos, la renta fija y el índice Dax
datos=pd.read_csv('DAX.csv',sep=';')

# datos.index=datos.Fecha
activos=datos.drop(['Fecha', 'Dax', 'BUND'], 1) # Extraemos los activos quitando la última columnasy la primera.
BUND=datos.loc[:, 'BUND'] # Extraemos la renta fija.
DAX=datos.loc[:, 'Dax'] # Extraemos el índice.

# Generamos las matrices de datos donde guardaremos los resultados.
rent_activos=pd.DataFrame(np.zeros((activos.shape[0] , activos.shape[1])), index=activos.index, columns=activos.columns)
rent_bund=pd.DataFrame(np.zeros(len(BUND)))
rent_DAX=pd.DataFrame(np.zeros(len(DAX)))
alpha_activos=pd.DataFrame(np.zeros((activos.shape[0] , activos.shape[1])), index=activos.index, columns=activos.columns)

# Calculamos la rentabilidad de los activos, renta fija e índice.
rent_activos = np.log(activos).diff()
rent_BUND = np.log(BUND).diff()
rent_DAX = np.log(DAX).diff()
rent_activos.iloc[:1]=0
rent_BUND[0]=0
rent_DAX[0]=0

# Calculamos la varianza del DAX
varianza_DAX = rent_DAX.rolling(10).var()

# Calculamos la covarianza del DAX y activos
cov_act_ind=pd.DataFrame(np.zeros((activos.shape[0] , activos.shape[1])), index=activos.index, columns=activos.columns)

for activo in activos.columns:
    cov_act_ind.loc[:,activo] = rent_DAX.rolling(10).cov(rent_activos.loc[:,activo])
    
beta=pd.DataFrame(np.zeros((cov_act_ind.shape[0] , cov_act_ind.shape[1])), index=cov_act_ind.index, columns=cov_act_ind.columns)

# Sacamos la Beta del activo. β=cov(Rc,Rm)/σRm
beta[10:beta.shape[0]] = cov_act_ind[10:rent_DAX.shape[0]].div(varianza_DAX[10:rent_DAX.shape[0]], axis=0)
alpha=pd.DataFrame(np.zeros((cov_act_ind.shape[0] , cov_act_ind.shape[1])), index=cov_act_ind.index, columns=cov_act_ind.columns)
Rfactivos=pd.DataFrame(np.ones((beta.shape[0] , beta.shape[1])), index=beta.index, columns=beta.columns)

 # Calculamos el Alpha del activo. α=Rc-(Rf+β(Rm-Rf))
alpha[10:alpha.shape[0]] = (rent_activos[10:rent_activos.shape[0]] - 
                            ((Rfactivos[10:Rfactivos.shape[0]].mul(rent_BUND[10:rent_activos.shape[0]], axis=0)) +
                            beta[10:beta.shape[0]].mul(rent_DAX[10:rent_activos.shape[0]] - rent_BUND[10:rent_activos.shape[0]], axis=0)
                            ))
                           
alpha=alpha[10:alpha.shape[0]]
tiempo_final = time() 
tiempo_ejecucion = tiempo_final - tiempo_inicial
 
print ('El tiempo de ejecucion fue:',tiempo_ejecucion ) #En segundos

alpha.head()

El tiempo de ejecucion fue: 0.12000012397766113


,Adidas,Allianz,Basf,Bayer,BEIERSDORF,BMW,COMMERZBANK,DAIMLER,DEUTSCHE BANK,DEUTSCHE BOERSE,...,LINDE,MAN,MERCK,METRO,MUNICHRE,RWE,SAP,SIEMENS,THYSSENKRUPP,VOLKSWAGEN
10,-0.011297,-0.015999,-0.001059,-0.021092,0.001689,-0.035989,-0.008341,-0.021082,-0.029637,-0.058729,...,0.014145,0.037230,-0.010452,-0.022294,-0.026809,0.003531,-0.024157,-0.001059,0.023966,-0.021699
11,0.009434,0.005770,-0.000529,0.019226,-0.007219,0.004917,0.006308,0.001671,0.017959,0.046029,...,0.016575,-0.021268,0.028489,0.005590,-0.002375,0.013146,-0.007424,-0.016964,0.021166,0.088198
12,-0.051190,-0.074325,-0.005452,-0.159888,-0.007898,-0.035220,-0.059276,-0.074048,-0.101870,-0.011080,...,-0.042639,-0.057515,-0.035034,-0.069675,-0.058894,-0.060233,-0.052089,-0.098766,-0.074753,-0.097590
13,0.015380,-0.035632,-0.002190,0.026103,0.003813,0.003324,-0.024555,-0.016557,-0.017729,-0.030303,...,0.015224,0.023463,0.019371,0.009771,-0.014863,-0.000771,-0.017187,-0.000832,0.003032,-0.007868
14,-0.014642,-0.000902,-0.001748,-0.043366,0.002927,0.003231,-0.051982,-0.017380,-0.027460,-0.021897,...,-0.013847,0.044897,-0.022620,-0.021749,-0.018192,-0.013314,-0.006052,0.008740,-0.002536,-0.019902


## Otra manera de hacerlo

In [3]:
import numpy as np
import pandas as pd
from time import time

In [4]:
def Alpha_Jensen():
    
    datos=pd.read_csv('DAX.csv' ,sep=';',parse_dates=True, index_col=0)
    num_activos=len(datos.columns)
    activos=datos.iloc[:,0:num_activos-1]
    BUND=datos.iloc[:,[-1]]
    DAX=activos.iloc[:,[-1]]
    rent_activos = np.log(activos).diff()
    rent_BUND = np.log(BUND).diff()
    rent_DAX = np.log(DAX).diff()
    varianza_DAX=rent_DAX.rolling(10).var()
    cov_act_ind = pd.DataFrame(columns=range(len(activos.columns)))
    
    for activo in range(len(activos.columns)):
        cov_act_ind.iloc[:,activo]=rent_activos.iloc[:,activo].rolling(10).cov(rent_activos.Dax)
        
    beta=cov_act_ind.iloc[:,:].div(varianza_DAX.Dax, axis=0)
    rent_esp=beta.mul(rent_DAX.iloc[:,:].sub(rent_BUND.BUND,axis=0).iloc[:,0],axis=0).add(rent_BUND.BUND,axis=0)
    rent_esp.columns=rent_activos.columns
    alpha=(rent_activos-rent_esp).dropna()
    
    return alpha

In [5]:
start_time = time()
alpha=Alpha_Jensen()
elapsed_time = time() - start_time
print("Elapsed time: %.10f seconds." % elapsed_time)

Elapsed time: 0.2859988213 seconds.


In [6]:
Alpha_Jensen()

,Adidas,Allianz,Basf,Bayer,BEIERSDORF,BMW,COMMERZBANK,DAIMLER,DEUTSCHE BANK,DEUTSCHE BOERSE,...,MAN,MERCK,METRO,MUNICHRE,RWE,SAP,SIEMENS,THYSSENKRUPP,VOLKSWAGEN,Dax
Fecha,,,,,,,,,,,,,,,,,,,,,
2003-01-15,-0.011297,-0.015999,-0.001059,-0.021092,0.001689,-0.035989,-0.008341,-0.021082,-2.963673e-02,-0.058729,...,0.037230,-0.010452,-0.022294,-0.026809,0.003531,-0.024157,-0.001059,0.023966,-0.021699,0.000000e+00
2003-01-16,0.009434,0.005770,-0.000529,0.019226,-0.007219,0.004917,0.006308,0.001671,1.795945e-02,0.046029,...,-0.021268,0.028489,0.005590,-0.002375,0.013146,-0.007424,-0.016964,0.021166,0.088198,3.469447e-18
2003-01-17,-0.051190,-0.074325,-0.005452,-0.159888,-0.007898,-0.035220,-0.059276,-0.074048,-1.018703e-01,-0.011080,...,-0.057515,-0.035034,-0.069675,-0.058894,-0.060233,-0.052089,-0.098766,-0.074753,-0.097590,2.602085e-18
2003-01-20,0.015380,-0.035632,-0.002190,0.026103,0.003813,0.003324,-0.024555,-0.016557,-1.772914e-02,-0.030303,...,0.023463,0.019371,0.009771,-0.014863,-0.000771,-0.017187,-0.000832,0.003032,-0.007868,0.000000e+00
2003-01-21,-0.014642,-0.000902,-0.001748,-0.043366,0.002927,0.003231,-0.051982,-0.017380,-2.746022e-02,-0.021897,...,0.044897,-0.022620,-0.021749,-0.018192,-0.013314,-0.006052,0.008740,-0.002536,-0.019902,1.734723e-18
2003-01-22,-0.015061,-0.026816,-0.000175,-0.068141,-0.012867,-0.032257,0.045622,-0.022991,-3.023963e-02,-0.020819,...,-0.033024,-0.013510,-0.056824,-0.037616,-0.030986,0.008841,-0.032796,-0.047080,-0.046562,0.000000e+00
2003-01-23,-0.035714,-0.000764,-0.002965,-0.023946,-0.024688,-0.055934,-0.060912,-0.028569,-8.966409e-03,-0.029200,...,-0.075799,0.007737,-0.014563,-0.013180,-0.034968,0.032965,0.049952,-0.031669,-0.045605,1.040834e-17
2003-01-24,-0.010078,-0.032636,-0.004951,-0.059248,-0.024717,-0.017495,-0.008992,-0.014840,-4.153725e-02,0.002153,...,-0.005749,0.019461,0.001764,-0.027566,-0.009941,-0.047614,-0.103886,-0.015370,-0.027411,8.673617e-19
2003-01-27,-0.027795,-0.041116,0.002689,-0.085556,-0.012160,-0.011741,-0.030295,-0.033903,-3.942808e-02,-0.033069,...,-0.009305,-0.029351,-0.083771,-0.060141,-0.013935,-0.014897,-0.065684,-0.026532,-0.025391,1.387779e-17


## Otra manera de hacerlo

In [7]:
import pandas as pd
import numpy as np
import time

In [8]:
ini = time.time()

ventana = 10
datos = pd.read_csv("DAX.csv", parse_dates=True, sep=";", index_col=0)
retornos = np.log(datos).diff().dropna()
dax = retornos["Dax"]
bund = retornos["BUND"]
activos = retornos.iloc[:, 0:30]
var = pd.DataFrame(dax.rolling(ventana).var().dropna())
cov = activos.rolling(ventana).cov(dax).dropna()
betas = pd.DataFrame(columns=cov.columns, index=cov.index)

for i in range(betas.shape[1]):
    betas.loc[:, betas.columns[i]]=cov.iloc[:, i] / var["Dax"]
    
alfa = pd.DataFrame(columns=betas.columns, index=betas.index)

for i in range(betas.shape[1]):
    alfa.loc[:, alfa.columns[i]]=activos.iloc[(ventana-1):, i] - (bund.iloc[(ventana-1):] + betas.iloc[:, i]*(dax.iloc[(ventana-1):] - bund.iloc[(ventana-1):]))

print(time.time()-ini)

0.32696032524108887


In [9]:
alfa

,Adidas,Allianz,Basf,Bayer,BEIERSDORF,BMW,COMMERZBANK,DAIMLER,DEUTSCHE BANK,DEUTSCHE BOERSE,...,LINDE,MAN,MERCK,METRO,MUNICHRE,RWE,SAP,SIEMENS,THYSSENKRUPP,VOLKSWAGEN
Fecha,,,,,,,,,,,,,,,,,,,,,
2003-01-15,-0.011297,-0.015999,-0.001059,-0.021092,0.001689,-0.035989,-0.008341,-0.021082,-2.963673e-02,-0.058729,...,0.014145,0.037230,-0.010452,-0.022294,-0.026809,0.003531,-0.024157,-0.001059,0.023966,-0.021699
2003-01-16,0.009434,0.005770,-0.000529,0.019226,-0.007219,0.004917,0.006308,0.001671,1.795945e-02,0.046029,...,0.016575,-0.021268,0.028489,0.005590,-0.002375,0.013146,-0.007424,-0.016964,0.021166,0.088198
2003-01-17,-0.051190,-0.074325,-0.005452,-0.159888,-0.007898,-0.035220,-0.059276,-0.074048,-1.018703e-01,-0.011080,...,-0.042639,-0.057515,-0.035034,-0.069675,-0.058894,-0.060233,-0.052089,-0.098766,-0.074753,-0.097590
2003-01-20,0.015380,-0.035632,-0.002190,0.026103,0.003813,0.003324,-0.024555,-0.016557,-1.772914e-02,-0.030303,...,0.015224,0.023463,0.019371,0.009771,-0.014863,-0.000771,-0.017187,-0.000832,0.003032,-0.007868
2003-01-21,-0.014642,-0.000902,-0.001748,-0.043366,0.002927,0.003231,-0.051982,-0.017380,-2.746022e-02,-0.021897,...,-0.013847,0.044897,-0.022620,-0.021749,-0.018192,-0.013314,-0.006052,0.008740,-0.002536,-0.019902
2003-01-22,-0.015061,-0.026816,-0.000175,-0.068141,-0.012867,-0.032257,0.045622,-0.022991,-3.023963e-02,-0.020819,...,-0.034409,-0.033024,-0.013510,-0.056824,-0.037616,-0.030986,0.008841,-0.032796,-0.047080,-0.046562
2003-01-23,-0.035714,-0.000764,-0.002965,-0.023946,-0.024688,-0.055934,-0.060912,-0.028569,-8.966409e-03,-0.029200,...,-0.015779,-0.075799,0.007737,-0.014563,-0.013180,-0.034968,0.032965,0.049952,-0.031669,-0.045605
2003-01-24,-0.010078,-0.032636,-0.004951,-0.059248,-0.024717,-0.017495,-0.008992,-0.014840,-4.153725e-02,0.002153,...,-0.038350,-0.005749,0.019461,0.001764,-0.027566,-0.009941,-0.047614,-0.103886,-0.015370,-0.027411
2003-01-27,-0.027795,-0.041116,0.002689,-0.085556,-0.012160,-0.011741,-0.030295,-0.033903,-3.942808e-02,-0.033069,...,-0.032479,-0.009305,-0.029351,-0.083771,-0.060141,-0.013935,-0.014897,-0.065684,-0.026532,-0.025391


## Otra manera de hacerlo

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
start_time = time.time()

ventana = 10

datos = pd.read_csv('DAX.csv', sep=';').set_index('Fecha')
returns = np.log(datos).diff()
columns = returns.columns.tolist()
rent_activos = returns[columns[:-2]]
rent_DAX = returns[['Dax']]
rent_BUND = returns[['BUND']]
var_DAX = rent_DAX.rolling(ventana).var()
cov_act_ind = pd.DataFrame(index=rent_activos.index)
for column in columns[:-2]:
    cov_act_ind[column] = rent_activos[column].rolling(ventana).cov(rent_DAX)
beta = cov_act_ind/var_DAX.values
alpha_activos = (rent_activos - (rent_BUND.values + beta*(rent_DAX.values - rent_BUND.values)))[ventana:]

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.155029296875 seconds ---


In [3]:
alpha_activos

,Adidas,Allianz,Basf,Bayer,BEIERSDORF,BMW,COMMERZBANK,DAIMLER,DEUTSCHE BANK,DEUTSCHE BOERSE,...,LINDE,MAN,MERCK,METRO,MUNICHRE,RWE,SAP,SIEMENS,THYSSENKRUPP,VOLKSWAGEN
Fecha,,,,,,,,,,,,,,,,,,,,,
15/01/2003,-0.011297,-0.015999,-0.001059,-0.021092,0.001689,-0.035989,-0.008341,-0.021082,-0.029637,-0.058729,...,0.014145,0.037230,-0.010452,-0.022294,-0.026809,0.003531,-0.024157,-0.001059,0.023966,-0.021699
16/01/2003,0.009434,0.005770,-0.000529,0.019226,-0.007219,0.004917,0.006308,0.001671,0.017959,0.046029,...,0.016575,-0.021268,0.028489,0.005590,-0.002375,0.013146,-0.007424,-0.016964,0.021166,0.088198
17/01/2003,-0.051190,-0.074325,-0.005452,-0.159888,-0.007898,-0.035220,-0.059276,-0.074048,-0.101870,-0.011080,...,-0.042639,-0.057515,-0.035034,-0.069675,-0.058894,-0.060233,-0.052089,-0.098766,-0.074753,-0.097590
20/01/2003,0.015380,-0.035632,-0.002190,0.026103,0.003813,0.003324,-0.024555,-0.016557,-0.017729,-0.030303,...,0.015224,0.023463,0.019371,0.009771,-0.014863,-0.000771,-0.017187,-0.000832,0.003032,-0.007868
21/01/2003,-0.014642,-0.000902,-0.001748,-0.043366,0.002927,0.003231,-0.051982,-0.017380,-0.027460,-0.021897,...,-0.013847,0.044897,-0.022620,-0.021749,-0.018192,-0.013314,-0.006052,0.008740,-0.002536,-0.019902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26/12/2011,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
27/12/2011,-0.002998,-0.009954,0.003113,-0.000116,0.000237,0.003608,-0.019915,-0.000986,-0.003121,0.016470,...,0.007433,0.008035,-0.015427,0.002580,0.004601,-0.006639,-0.002373,0.002100,0.010732,-0.000050
28/12/2011,-0.012136,-0.032753,-0.016857,-0.020900,-0.007337,-0.034674,-0.041400,-0.040033,-0.001666,-0.023234,...,-0.008256,-0.014007,0.002509,-0.023816,-0.024575,-0.040576,-0.019171,-0.012240,-0.027933,-0.027818
